Let us setup an interactive `Python` session where we can run the code
developed here.

``` jupyter
print("Welcome to EMACS Jupyter")
```

We develop topological analyses of brain circuits, and implement
`Python` code that we can use with to configure a `connsense-pipeline`.
Instead of the connectivity of the whole circuit *Mosaic*, our focus
will be `circuit-subtargets` that are geometrically defined. We will
start by describing how to configure a `connsense-pipeline`, run it, and
use `connsensnse-tap` to access and analyze the data.

# Setup

In our discussion we will develop scientific concepts to measure the
circuit, and implement Python functions to compute them. Here we setup a
notebook template to test and explore, and the structure of a `Python`
package for our methods.

To get the notebook you will have to clone,

``` shell
git clone https://bbpgitlab.epfl.ch/conn/structural/topological-analysis-of-subvolumes.git
git checkout beta
```

In [ ]:
from importlib import reload
from collections.abc import Mapping
from collections import OrderedDict
from pprint import pprint, pformat
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib

reload(matplotlib)
from matplotlib import pylab as plt
import seaborn as sbn

from IPython.display import display

from bluepy import Synapse, Cell, Circuit

GOLDEN = (1. + np.sqrt(5.))/2.
print("We will plot golden aspect ratios: ", GOLDEN)


``` example
We will plot golden aspect ratios:  1.618033988749895
```

## Workspaces

We have run `connsense-CRAP` for the SSCx dissemination variant *Bio-M*,
extracting data that we will use to compute the factology. Here is a
list of workspaces we will need to generate factsheets.

In [ ]:
ROOTSPACE = Path("/")
PROJSPACE = ROOTSPACE / "gpfs/bbp.cscs.ch/project/proj83"
SOODSPACE = PROJSPACE / "home/sood"
CONNSPACE = SOODSPACE / "topological-analysis-subvolumes/test/v2"
DEVSPACE  = CONNSPACE / "test" / "develop"


## `connsense` Modules

While test-developing it will be good to have direct access to the
`connsense-TAP-store` we will use. We will use a module from `connsense`
to load the HDFstore,

In [ ]:
from connsense.develop import topotap as cnstap
tap = cnstap.HDFStore(CONNSPACE/"pipeline.yaml")
circuit = tap.get_circuit("Bio_M")
print("Available analyses: ")
pprint(tap.analyses)
circuit


# Introduction

We will study flatmap columns. In the circuit's flatmap we have the
following grid,

In [ ]:
from flatmap_utility import subtargets as fmst
fmst.fmap_positions(circuit, over_flatmap_voxels=None, with_orientations=None,
                    to_subpixel_resolution=True, dropna=True)


In [ ]:
from flatmap_utility import subtargets as fmst, tessellate
flat_xys = fmst.fmap_positions(in_data=circuit)
tritille = tessellate.TriTille(230.0)
graphic_fmap_cells = tritille.plot_hextiles(flat_xys,
                                            annotate=False, with_grid=False, pointmarker=".", pointmarkersize=0.05)
graphic_fmap_cells[0]


``` example
/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/proj83/topological-analysis-of-subvolumes/flatmap_utility/flatmap_utility.py:199: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  res = Rotation.align_vectors(vtgt, vv)
Rotation errors: min: 0.0, median: 0.09387602600937707, mean: 0.1362824184485066, std: 0.15664142313770807, max: 2.0
```

![](attachment:./figures/flatmap.png)

# Pipeline: TODO

# TAP CLI

# Subtargets

We have information about subtargets in `tap`,

In [ ]:
display(tap.subtargets)


Each one is a straight up cylinderical column in the flatmap-space, and
a conical column in circuit's physical space. Each `subtarget` is
associated with a list of gids,

In [ ]:
fig_subtarget_sizes = plt.figure(figsize=(GOLDEN * 12, 12))

ax = sbn.histplot(tap.subtarget_gids.apply(len).values, bins=25,
                  ax=fig_subtarget_sizes.add_subplot())


We have `subtargets`' `conicality, volume, and height` in the `info`. We
may not be able to compute these values for each `flatmap-column`. Some
may have a geometry that does not resolve to our computations. The `nan`
values for `conicality` are found in,

In [ ]:
flat_xy = OrderedDict([("flat_x", "x"), ("flat_y", "y")])
nanxys = tap.subtargets[list(flat_xy.keys())].rename(columns=flat_xy)

graphic_nanfmap = tritille.plot_hextiles(nangxys, annotate=False)
graphic_nanfmap[0]


``` example
[0;31m---------------------------------------------------------------------------[0m
[0;31mNameError[0m                                 Traceback (most recent call last)
[0;32m/tmp/ipykernel_28068/3521415563.py[0m in [0;36m<module>[0;34m[0m
[1;32m      2[0m [0mnanxys[0m [0;34m=[0m [0mtap[0m[0;34m.[0m[0msubtargets[0m[0;34m[[0m[0mlist[0m[0;34m([0m[0mflat_xy[0m[0;34m.[0m[0mkeys[0m[0;34m([0m[0;34m)[0m[0;34m)[0m[0;34m][0m[0;34m.[0m[0mrename[0m[0;34m([0m[0mcolumns[0m[0;34m=[0m[0mflat_xy[0m[0;34m)[0m[0;34m[0m[0;34m[0m[0m
[1;32m      3[0m [0;34m[0m[0m
[0;32m----> 4[0;31m [0mgraphic_nanfmap[0m [0;34m=[0m [0mtritille[0m[0;34m.[0m[0mplot_hextiles[0m[0;34m([0m[0mnangxys[0m[0;34m,[0m [0mannotate[0m[0;34m=[0m[0;32mFalse[0m[0;34m)[0m[0;34m[0m[0;34m[0m[0m
[0m[1;32m      5[0m [0mgraphic_nanfmap[0m[0;34m[[0m[0;36m0[0m[0;34m][0m[0;34m[0m[0;34m[0m[0m
```

:

``` example
[0;31mNameError[0m: name 'tritille' is not defined
```

<img src="attachment:./figures/nan_flatmap.png"
style="width:95.0%;height:95.0%" />

# Nodes

We extract node properties as `extract-node-populations/default`, and
can access the node-properties using `tap`,

In [ ]:
nodes = cnstap.TapDataset(tap, ("extract-node-populations", "default"), belazy=False)


As you can tell from the numbers that the `nodes` dataframe contains all
the nodes in all the subtargets (*i.e* all circuit nodes except those
that could not be flatmapped). This dataframe is no different from the
`circuit.cells` one, except that the `subtarget` is avaialble in the
index,

In [ ]:
display(nodes.frame[ ["gid", "layer", "mtype"] ].head())


Notice the `node_id` which is not the same as `gid`. The `node_id` value
is local to the `subtarget`.

# Adjacency matrices

Adjacency matrices for edges in the local connectome were extracted as
`extract-edge-populations/local`. This data is stored as sparse
matrices, and hence using `.frame` on the non-lazy data not make much
sense. The data is also much larger than nodes. We should stick to the
lazy `datacalls` loaded by `tap`,

In [ ]:
display(tap.adjacency.frame.head())


Each entry can be expanded when needed,

In [ ]:
adj = tap.adjacency.dataset.iloc[0]()

print("matrix of type", type(adj), "and shape", adj.shape,
      "number of connections", adj.astype(bool).sum())


# Analyses

## Simplex counts

We already have results for simplex-counts in a `tap-store` resulting
from a run of the \~connsense-pipeline- developed in <span
class="spurious-link" target="Pipeline">*Pipeline*</span>. Let us load
the simplex-counts,

In [ ]:
simplex_counts = cnstap.TapDataset(tap, ("analyze-connectivity", "simplex-counts"),
                                   belazy=False)
print("Simplex counts as a %s"%(type(simplex_counts),))


We have run simplex counts for both the full (original and 5 ER
controls) adjacency matrices, and their slices by layer. We can load the
results for the `full-flatcolumns` as a dataframe,

In [ ]:
simplex_counts.frame["full"]


This computation will take a while, as all the simplex counts, for both
the full and layer will be decompressed from `connsense.h5` and loaded
into a `pandas.DataFrame`. In about a minute you should see the result.
Once loaded, the dataframe will stay with the object
`simplex_counts.frame`. For layers we have an extra level in the index,

In [ ]:
simplex_counts.frame["layer"]


We find a `nan` where no simplices were found. Each individual
computation on a single `flatcolumn` returns an array / series of
simplex counts by dimension *upto* the maximum dimension of simplices
found in that `flatcolumn`. When we concatenated these data into a
single `frame`, the max-dimension was set to the maximum of maxima over
all `flatcolumns` (and their randomizations). This filled in `nan`
values for `flatcolumns` that did not have the higher missing
dimensions. While we can replace the `nans` by zeros, as that is what
they mean, they do offer a convenient way to compute the max-dimension,

In [ ]:
from tqdm import tqdm; tqdm.pandas()

maxdim = (simplex_counts.frame["full"]
          .apply(lambda subtarget: len(subtarget.dropna()), axis=1)
          .rename("dim"))

display(maxdim)


which we can convert to a plot,

In [ ]:
fig_maxdim = plt.figure(figsize=(GOLDEN * 8, 8))
_ax = sbn.violinplot(x="control", y="dim", data=maxdim.reset_index(),
                     ax=fig_maxdim.add_subplot())
fig_maxdim


What about each layer?

In [ ]:
maxdim_by_layer = (simplex_counts.frame["layer"]
                   .apply(lambda subtarget: len(subtarget.dropna()), axis=1)
                   .rename("dim"))

display(maxdim_by_layer)


and it looks like,

In [ ]:
fig_maxdim_by_layer = plt.figure(figsize=(GOLDEN * 8, 8))
_ax = sbn.violinplot(x="control", y="dim", hue="layer", data=maxdim_by_layer.reset_index(),
                     ax=fig_maxdim_by_layer.add_subplot())
fig_maxdim_by_layer


## Model Params DD2

Loading the results for second order distance dependent model is similar
to simplex counts,

In [ ]:
params_dd2 = cnstap.TapDataset(tap, ("analyze-connectivity", "model-params-dd2"),
                               belazy=False)
display(params_dd2.frame)


## Simplices

We would like to analyze higher order statistics of simplices than the
humble simplex counts. We could save simplices as lists for each
subtarget input. However this data can be large. To reduce the computed
data's size, we can compute statistics on each `subtarget`'s simplices,

In [ ]:
def node_participation(simplices):
    """Compute node participation from DataFrame of simplices,
    containing a list of simplices in each row.
    """
    return simplices.apply(lambda column: column.value_counts(), axis=0).fillna(0).astype(int)


or for edge participation,

In [ ]:
def edge_participation(simplices):
    """Compute node participation from DataFrame of simplices,
    containing a list of simplices in each row.
    """
    edge_positions = list(range(simplices.shape[1]))[:-1]

    def value_count_edge(position):
        counts = simplices[ [position, position+1] ].value_counts().rename(position)
        counts.index.rename(["source", "target"], inplace=True)
        return counts

    return pd.concat([value_count_edge(p) for p in edge_positions], axis=1).fillna(0).astype(int)


# Scratch